# Introduction

SSH and SCP are network protocols by which two computers can securely communicate, even over unsecured networks. SSH is primarily used for remote log-in (i.e., logging into one computer remotely from another computer) and remote command line control (i.e. using one computer to open a terminal on another). SCP is a related protocol for copying files from one computer to another via the SSH protocol. Both are command line tools, which can be accessed from a terminal on Linux or MacOS, and via Windows on a number of services which provide SSH functionality, i.e. [MobaXterm](https://mobaxterm.mobatek.net/). Both of these services allow one to remotely work on a computer without physically being present at that computer. This is often used to access a computing cluster, such as NERSC, but can be used to connect to any computer. 

For this notebook, don't execute the code blocks in the python notebook, as they won't do anything. Do copy and paste them into your terminal, making the appropriate changes, e.g. inserting your actual user name for *user*.

## Basic SSH


SSH-ing into a computer is generally used to create a terminal running on that remote computer but which is accessible from the local computer. So, for example, after SSH-ing into a cluster from my laptop, I will have a terminal on my laptop which has access to the file systems of the cluster, and which will execute commands on that cluster. To be even more explicit, if I type:


<code>ls</code>

on the SSH terminal on my laptop, it will show me my home directory on the cluster, and further if I type:

<code>python some_script.py</code>

It will run some_script.py, which is located on the cluster, using the cluster resources. Note generally you should submit jobs on a cluster rather than running them like this but for the purposes of this example that’s besides the point. 

To SSH into a computer, first open a terminal on Linux or MacOS, or on Windows start MobaXterm and click “Start local terminal”. The general syntax for SSH is:


<code>ssh user@remote.host.edu</code>

Where user is the username on remote.host.edu, the computer you are trying to SSH into. You can also SSH via the IP address, i.e.:

<code>ssh user@185.52.53.222</code>

Go ahead and try connecting to the account you set up on Jeff's laptop via:

<code>ssh studentX@10.66.46.147</code>

where studentX is the account that was made for you. Generally, the first time you connect to a remote host SSH will ask you if you trust the host you are connecting to. Assuming you do, go ahead and enter yes. You will also be asked for a password if one is associated with your user. The password for your account is PennSummerx; enter it. Note that generally no characters will display when typing in an SSH password.

There are many options you will commonly pass to SSH. A full listing of them can be found in the SSH manual. Note that capitalization matters for opinions, i.e. <code>-p</code> is different than <code>-P</code>. Some common ones you’ll use include:


1. <code>-Y</code>: enables X11 forwarding, which allows you to view images opened on the remote server on your local computer.

2. <code>-i</code>: private_key: specifies a private key, in this case a file called private_key, to use for the session: see the section ‘Setting up Keys’

3. <code>-p</code>: 12345: allows you to specify a port, e.g. in this case 12345
4. <code>-v</code>: verbose. Causes SSH to print verbose debugging messages while working. Can be useful for debugging connections that aren’t working correctly. 

## Setting up Keys


SSH can be configured to use a public/private keypair instead of or in addition to a password to secure your connection. Many clusters will require you to use a keypair to login. A keypair consists of a public key, which you can freely disseminate, and a private key, which you keep on the computer from which you will be logging in. The public key can be used by anyone to encode a message, while only you posses the private key needed to decode it. To use a keypair, you first need to generate a key. To do so, on a terminal (or a MobaXterm terminal), do:


<code>ssh-keygen -t ed25519</code>


his will ask you for a place to save the key, as well as a password for the key which will allow you to use it. The default save location is


<code>${HOME}/.ssh/</code>

The private key will be saved as 

<code> ${HOME}/.ssh/id_type</code>

where in this case id_type is ed25519, while the public key will be saved as

<code> ${HOME}/.ssh/id_type.pub </code>

Go ahead and use this location and file name, and enter whatever secure password you choose. If you know that you are going to have to create keys for a number of clusters, you’ll probably want to change the name of the keys, perhaps to something like clustername_ed22519. An example terminal output, courtesy of compute Canada, will look like:


<code>ssh-keygen -t ed25519
Generating public/private ed25519 key pair.
Enter file in which to save the key (/home/USERNAME/.ssh/id_ed25519): 
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /home/USERNAME/.ssh/id_ed25519.
Your public key has been saved in /home/USERNAME/.ssh/id_ed25519.pub.
The key fingerprint is:
SHA256:EajOndriRmLpl1qKg03FDhnc0EzRaApdBTygEbpQZrA USERNAME@HOSTNAME
The key's randomart image is:
+--[ED25519 256]--+
|+=*X=*...        |
|oB+ O o  .       |
|E. * o  .        |
|..+ +    .       |
|.  B . .S        |
|  = = o          |
|.= o.+           |
|o.oo* .          |
|..o=..           |
+----[SHA256]-----+
</code>

**NEVER COPY YOUR PRIVATE KEY TO ANOTHER COMPUTER.**

Copying the key greatly increases the security risk that the key is stolen. For each computer from which you wish to SSH from, generate a new key pair.

Once you have your key pair, you’ll need to upload the public key to the computer you wish to access. The details of this process will vary from one cluster to another, so you’ll have to do a bit of your own investigation. Generally you’ll simply upload the plain text of the public key to a website. To see the plain text of the public key, do:



<code>cat ~/.ssh/id_ed25519.pub</code>

You should now be able to SSH using your keypair using the <code>-i</code> flag. Note that if you used the default location and name, using the <code>-i</code> flag is not necessary. The usage is:


<code>ssh -i /path/to/key user@remote.host.edu</code>

It will then prompt you for the password; note here it is referring to the password for the *private key*, which you just set up, and not any password you may have set up for your account at <code>remote.host</code>.


## SCP

SCP is a protocol for securely transferring files from one computer to another. SCP works very similarly to SSH, except that we specify a file to be transferred in addition to the address of the remote computer. SCP is always used to transfer files from a local computer to a remote computer, not between two remote computers. That being said the files can be sent either from the local computer to the remote computer or vice-versa.  Additionally, transfer between two remote computers is possible by combining SCP and SSH. Firstly, to transfer a file from your local computer to a remote computer, do:

<code>scp /local/path/to/foo.txt user@remote.host.edu:/path/to/remote/dir</code>

Where the first argument, <code>/local/path/to/foo.txt</code> is the location of the file foo.txt on our local computer we want to move, <code>user@remote.host.edu</code> is our username at the remote computer, and <code>/path/to/remote/dir</code> is where we want to copy foo.txt on the remote computer. 

To transfer files from a remote computer to our local machine, we essentially just reverse the arguments in the above, i.e.



<code>scp user@remote.host.edu:/path/to/remote//foo.txt  /local/path/to/dir/</code>


Although note that the file is still specified in the first argument. If an ssh keypair is in use with this pair of machines, simply specify the private key location with <code>-i</code>, identically to with ssh:


<code>scp -i /path/to/key /local/path/to/foo.txt user@remote.host.edu:/path/to/remote/dir</code>

Finally, to SCP transfer files between two remote computers, first SSH into one of the computers using the <code>-A</code> flag:

<code>ssh -A user@remote.host.edu</code>

Then SCP as usual:

<code>scp local/path/to/foo.txt user@another.host.edu:/path/to/remote/dir</code>

### rsync / sftp 

These work similar to the predescribed scp, with some useful differences. 


`rsync` synchronises files between the two endpoints. It functions similarly to scp - you tell it where you want to transfer the file(s) from and to. Then, rsync will just copy the parts of the file(s) that are different, saving time on large files. Adding `-aP` adds a neat progress bar. Example:

`rsync -aP "user@cluster.location:/home/user/location/of/files/*" /local/machine/directory/`

Note the quotation marks around the cluster address. This is necessary for iTerm or Mac M1s (I forget which) to correctly read in that entire string for the cluster location.


`sftp` stands for SSH file transfer protocol. It's a little different from rsync and scp. This lets you SSH into a remote machine and move around in its directories as well as local directories and simply get or put files in either location. It's great for moving files around both ways in different locations. Example:

`ssh user@cluster.location`

`cd some/remote/directory`

`get some_remote_file`

`lcd some/local/dorectory`

`put some_local_file`

Here, cd is change remote directory while lcd is local change directory, same for pwd and lpwd. The command get downloads a remote file to the local working directory, while put uploads a local file to the remote working directory. 

## NERSC

If your PI wants you to perform analysis on NERSC, you should read this section. Applying for a NERSC user account is covered on their website [here](https://docs.nersc.gov/accounts/#obtaining-an-account), but as a summary you'll first need to ask your PI for the name of the project you will be assigned to on NERSC project (names are often the letter "m" followed by some (usually four) digits, e.g., m1234). Once you have that, apply for an account using [this form](https://iris.nersc.gov/add-user). That portal, [iris.nersc.gov](iris.nersc.gov), is the website you will use to manage your NERSC account, although not to connect to NERSC. The application process can take up to a week; you will receive an email when it is completed. Assuming your application was accepted, you'll recieve a temporary password to log into iris and set up a perminant password. The temporary password will only last for 24 hours so don't delay in logging in!

You should also set up multi-factor authenication (MFA) using the iris portal. This process is detailed well at [this NERSC page](https://docs.nersc.gov/connect/mfa/); I strongly recommend using the Google Authenticator app, as it's the best supported. MFA is second password for your account, which frequently changes. Your current MFA password will be displayed in the Google Authenticator app once set up; they change every 15 seconds or so. You'll have to enter both your regular password and current MFA password to access your NERSC account.

Once your account is set up, I find the best way to access NERSC is their [Jupyter portal](https://jupyter.nersc.gov/hub/home). This portal includes not only Jupyter notebooks but also the ability to open terminals, and allows you to do most everything you could need to do. 

